In [40]:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples

In [42]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

def process_tweets(tweet):
    #remove the old style retweet text "RT"
    tweet2 = re.sub( r'^RT[\s]+' , '' ,tweet)

    #remove hyperlinks
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*' , '' ,      tweet2)

    #remove hashtag (only the # sign)
    tweet2 = re.sub(r'#' , '' ,tweet2)

    #instantiate the tokenizer class
    tokenizer = TweetTokenizer  (preserve_case=False,
                          strip_handles=True,
                          reduce_len = True)

    #tokenize tweets
    tweet_tokens = tokenizer.tokenize(tweet2)

    #importing the english stop words from nltk
    stopwords_english = stopwords.words('english')

    tweets_clean = []

    for word in tweet_tokens:
        if(word not in stopwords_english 
        and 
        word not in string.punctuation):
            tweets_clean.append(word)
    
    stemmer = PorterStemmer()

    #create an empty list to store the stems
    tweets_stem = []

    for word in tweets_clean:
        stem_word = stemmer.stem(word)  
        #stemming word
        tweets_stem.append(stem_word)
    
    return tweets_stem

In [43]:
def build_freqs(tweets , ys):
    """Build frequencies
    input:
        tweets : a list of tweets
        ys: an mx1 array with the sentiment label of each tweet(either 0 or 1)
    output:
        freqs: a dictionary mapping each (word,sentiment) pair to its frequency
        """
    
    #convert the np array to list since zip needs an iterble
    # The squeeze is necessary or the list ends up with  one element
    # also note that this is just a NOP if ys is already a list
     
    yslist = np.squeeze(ys).tolist()

    #start with an empty dictionary and populate it by looping  over all tweets
    # and over all processed words in each tweet
    freqs={}

    for y , tweet in zip(yslist , tweets):
        for word in process_tweets(tweet):
            pair =  (word,y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

In [44]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [45]:
# training and testing split
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg


In [46]:
#combine positive and negative labels
train_y = np.append(np.ones(( len(train_pos),1 )) , np.zeros((len(train_neg),1)),axis =0 )
test_y = np.append(np.ones(( len(test_pos),1 )) , np.zeros((len(test_neg),1)),axis =0 )

In [47]:
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [48]:
# create frequency duictionary
freqs = build_freqs(train_x,train_y)

print("type(freqs) = "+str(type(freqs)))
print("len (freqs) = "+str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len (freqs) = 11339


In [49]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweets(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


# PART 1 LOGISTIC REGRESSION

## part 1.1 sigmoid

In [50]:
# UNQ_C1
# code to be submitted
def sigmoid(z):
    import math
    h = 1 / (1 + math.exp(-z) )
    return h

In [51]:
# Testing your function 
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

SUCCESS!
CORRECT!


## part 1.2 cost function and gradient

In [52]:
# log(0) -> log(1) == -inf -> 0
# h(z)for ith example if equal to y gives 0
# if they are not then log gives a big positive value

# verify that when the model predicts close to 1, but the actual label is 0, the loss is a large positive value
#lets take y 0.99 and hz as 0
-1*( (1*np.log(0.0001)) + (1-1)*(np.log(1-0.0001)) )

9.210340371976182

In [53]:
# verify that when the model predicts close to 0 but the actual label is 1, the loss is a large positive value
-1 * np.log(0.0001) # loss is about 9.2


9.210340371976182

In [54]:
def gradientDescent(x,y,theta,alpha,num_iters):
    '''
    Input:

        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions(m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down
    '''
    m = len(x)

    for i in range(0,num_iters):

        z = np.dot(x,theta)
        newfunc = np.vectorize(sigmoid)
        h = newfunc(z)

        term1 = np.dot(np.transpose(y),np.log(h))
        term2 = np.dot( np.transpose(1-y),np.log(1-h) )
        J = - (1/m) * (term1 + term2)

        theta = theta - (alpha/m)*np.dot(np.transpose(x),(h-y))
    
    J = float(J)
    return J,theta

In [55]:
# Check the function
# Construct a synthetic test case using numpy PRNG functions
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]


# part 2 : extracting features

In [56]:
# UNQ_C3
def extract_features(tweet,freqs) :
    '''
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary coreesponding to the frequencies of each tuple (word,label)
    Output:
        x: a feature vector of dimension(1,3)
    '''

    word_1 = process_tweets(tweet)

    x = np.zeros((1,3))

    x[0,0] = 1

    for word in word_1:
        pair_pos = (word,1)
        pair_neg = (word,0)

        if pair_pos in freqs:
            x[0,1] += freqs[pair_pos]
        else:
            x[0,1] += 0

        if pair_neg in freqs:
            x[0,2] += freqs[pair_neg]
        else:
            x[0,2] += 0
        
    
    assert(x.shape==(1,3))
    return x


In [57]:
# Check your function

# test 1
# test on training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


In [58]:
 #test 2:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


# part 3 traning model

In [59]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.24216529.
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]


# part 4 test your logistic regression

In [68]:
#UNQ_C4

def predict_tweet(tweet,freqs,theta):
    '''
    input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word,label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive and negative
    '''

    x = extract_features(tweet,freqs)

    y_pred = sigmoid(np.dot(x,theta))

    return y_pred


In [69]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

I am happy -> 0.518580
I am bad -> 0.494339
this movie should have been great. -> 0.515331
great -> 0.515464
great great -> 0.530898
great great great -> 0.546274
great great great great -> 0.561561


In [79]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

0.8163648751301829

## check performance using the test set

In [88]:
def test_logistic_regression(test_x,test_y,freqs,theta):
    y_hat = []

    for tweet in test_x :
        y_pred = predict_tweet(tweet,freqs,theta)

        if y_pred > 0.5 :
            y_hat.append(1.0)
        else:
            y_hat.append(0)
        
    accuracy = (y_hat == np.squeeze(test_y)).sum() / len(test_x)
    return accuracy

In [89]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950


# part 5 error analysis

In [93]:
# Some error analysis done for you
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweets(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweets(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
THE TWEET IS: @jaredNOTsubway @iluvmariah @Bravotv Then that truly is a LATERAL move! Now, we all know the Queen Bee is UPWARD BOUND : ) #MovingOnUp
THE PROCESSED TWEET IS: ['truli', 'later', 'move', 'know', 'queen', 'bee', 'upward', 'bound', 'movingonup']
1	0.49996902	b'truli later move know queen bee upward bound movingonup'
THE TWEET IS: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
THE PROCESSED TWEET IS: ['sure', 'would', 'good', 'thing', '4', 'bottom', 'dare', '2', 'say', '2', 'miss', 'b', 'im', 'gonna', 'stubborn', 'mouth', 'soap', 'nothavingit', ':p']
1	0.48622918	b'sure would good thing 4 bottom dare 2 say 2 miss b im gonna stubborn mouth soap nothavingit :p'
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
THE PROCESSED TWEET IS: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48370688	b"i'm play brain dot braindot"
THE TWEET IS: I'm p

# part 6 predict with your own tweet

In [104]:
# Feel free to change the tweet below
my_tweet = 'My disappointment knows no bound and my day is ruined'
print(process_tweets(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['disappoint', 'know', 'bound', 'day', 'ruin']
0.506990608084016
Positive sentiment
